In [1]:
import json

In [103]:
dates=['2021-06-01','2021-09-01','2021-12-01','2022-03-01','2022-06-01']

In [130]:
array = []

for date in dates:
    
    with open('posts_'+date+'.json', 'r') as f:
        data = json.load(f)
        
        comment_no = 0
        # "comments","t1_hmtrrkz", "media", "richtextContent", "document", "c"
        for key in data:
            if key == "comments":
                for comment in data[key]:
                    row=[]
                    row.append(date)
                    for i in range(0,7):
                        try:
                            value=data[key][comment]['media']["richtextContent"]["document"][0]['c'][i]['c'][0]['c'][0]['t']
                            print(value)
                            row.append(value)
                        
                        except:
                            try: 
                                value=data[key][comment]['media']["richtextContent"]["document"][1]['c'][i]['c'][0]['c'][0]['t']
                                print(value)
                                row.append(value)
                            except:
                                try: 
                                    value=data[key][comment]['media']["richtextContent"]["document"][i]['c'][0]['t']
                                    print(value)
                                    row.append(value)
                                except:
                                    pass
                    if len(row)>3:
                        array.append(row)
                    comment_no += 1

# for i in range(0,5):
#     try:
#         print(data["comments"]["t1_hmtrrkz"]['media']["richtextContent"]["document"][0]['c'][i]['c'][0]['c'][0]['t'])
#     except:
#         pass

Data Pipeline - Business Side, Sr. Manager
15+ years
LCOL, US
150K
25%
This sounds like a great job. Are you a product manager?
Yes.  And the data / data system is my product.
What is your title? I’m thinking about switching to product manager
Data Engineer
2.5 years as a DE (3 months of that was an internship)
100% Remote; company is based out of a low cost of living city in the southern part of US
130,000USD
10% annual bonus (among other benefits)
Damn that’s nice.
[deleted]
You'd probably want to look for distributed companies, but it seems like a lot are readjusting costs based on location, which sucks :(
[deleted]
Ohhh, based out of Europe makes a bit more sense. They seem to pay lower than those based in the US, but I can't confirm that.
Tech stack is standard: AWS, Python, Snowflake, Looker, and some various databases hosted on AWS. I spend most of my time in Snowflake and Python.
Data Engineering Consultant
0.5 years
London
£32K
Not to be rude but isn’t this too low? Or is this

In [131]:
import pandas as pd
df=pd.DataFrame(array)

df.to_csv('salary_data.csv', index=False)



In [132]:
df

,0,1,2,3,4,5,6,7
0,2021-06-01,"Data Pipeline - Business Side, Sr. Manager",15+ years,"LCOL, US",150K,25%,None,None
1,2021-06-01,Data Engineer,2.5 years as a DE (3 months of that was an int...,100% Remote; company is based out of a low cos...,"130,000USD",10% annual bonus (among other benefits),None,None
2,2021-06-01,Data Engineering Consultant,0.5 years,London,£32K,None,None,None
3,2021-06-01,"It's his first job presumably, I had the same ...",It was tough to live though.,Work doesn't really pay in Europe like it does...,None,None,None,None
4,2021-06-01,ETL Developer,1.5 years,Belarus,$15k USD,None,None,None
...,...,...,...,...,...,...,...,...
330,2022-06-01,Senior Data Engineer,"5, coming on 6",Seattle (but remote mainly),$165k base,"Annual bonus ($25k signing bonus), some equity...",Media,"AWS, Airflow, Spark, some other DevOps tools"
331,2022-06-01,Data Engineer,"1 in role, 2 months in DS internship about 6 m...",Virginia (hcol),102k USD,"Paltry bonus- 10% 401k contrib, full healthcar...",Government,"Python, sql, unfortunately no cloud resources lol"
332,2022-06-01,Data Engineer,8 Years as a BI Dev 2 Years Data Engineer,London UK,"80,000 GBP before Tax",Upto 15% Bonus,Asset Management,"SQL, Azure Data Factory, Python (just a little..."
333,2022-06-01,title: Data Engineer 1,location: Michigan (remote CA company),pay: 145k base,bonus: 10% base/yr + 10k rsu sign on,industry: cannabis,"stack/libs: Python, Pandas, postgresql, boto3,...",None


In [120]:
df

,0,1,2,3,4,5
0,2021-06-01,"Data Pipeline - Business Side, Sr. Manager",15+ years,"LCOL, US",150K,25%
1,2021-06-01,Data Engineer,2.5 years as a DE (3 months of that was an int...,100% Remote; company is based out of a low cos...,"130,000USD",10% annual bonus (among other benefits)
2,2021-06-01,Data Engineering Consultant,0.5 years,London,£32K,None
3,2021-06-01,ETL Developer,1.5 years,Belarus,$15k USD,None
4,2021-06-01,Sr Data Engineer,7 years professional experience. 2 in data eng...,Los Angeles,"$125,000 base salary","$12,500 bonus target + $10,000 RSU/year"
...,...,...,...,...,...,...
242,2022-06-01,Data Analyst,3 years,Alberta,$90k CAD,"Negligible bonus, 2% match RRSP"
243,2022-06-01,Senior Data Engineer,"5, coming on 6",Seattle (but remote mainly),$165k base,"Annual bonus ($25k signing bonus), some equity..."
244,2022-06-01,Data Engineer,"1 in role, 2 months in DS internship about 6 m...",Virginia (hcol),102k USD,"Paltry bonus- 10% 401k contrib, full healthcar..."
245,2022-06-01,Data Engineer,8 Years as a BI Dev 2 Years Data Engineer,London UK,"80,000 GBP before Tax",Upto 15% Bonus


In [38]:
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

In [41]:
with open('posts.json', 'r') as f:
    data = json.load(f)
    #print(data)
    my_data= json_extract(data.json(), 'comments')
    print(my_data)

AttributeError: 'dict' object has no attribute 'json'

[]